# ATA control

`ATATools.ata_control` is a Python library that can be used to control several aspects of the Allen Telescope Array, such as the pointing of the antennas, the focusing of the feeds, the LO frequency of the RFCB, and the IF switch that selects the antennas that are connected to the GNU Radio backend USRPs.

`ata_control` is very useful for interactive, real time control of the array, as well as for writing Python scripts that can perform complex observations.

`ata_control` is included in the [ATA-Utils](https://github.com/SETIatHCRO/ATA-Utils) repository.

Here we give some examples of its usage. This notebook can be used as a cheat-sheet for interactive control.

## GNU Radio backend at the ATA

This diagram describes the structure of the GNU Radio backend at the Allen Telescope Array.

![gnuradio backend diagram](gnuradio-backend-diagram.png "GNU Radio backend diagram")

In [ ]:
from ATATools import ata_control as ac
# Angle from astropy is useful if we want to specify pointing
# coordinates in hours-minutes-seconds and degrees-minutes-seconds
from astropy.coordinates import Angle
import urllib.request  # Used to download data

## Checking the array status

Before we begin, it is useful to check what the array is doing to have an idea of what are the antennas in use, etc. We can also use this later on to check that our antennas are pointing correctly.

In [ ]:
print(ac.get_ascii_status())

## Reserving the antennas

It is important to reserve the antennas we will use to prevent clashes with other users of the system. The function below will fail if some of the antennas are already reserved, stopping automated scripts before the step on someone elses's observation.

In [ ]:
antennas = ['1a', '1f', '5c']  # List of antennas that we want to use (the USRPs can use 2 antennas at a time only)
ac.move_ant_group(antennas, 'none', 'atagr')  # Reserve the antennas

In [ ]:
ac.list_antenna_group('atagr')  # List antennas reserved by 'atagr'

In [ ]:
ac.list_antenna_group('none')  # List free antennas antennas

## Selecting the antennas in the IF switch

The IF switch is used to select what antennas are routed to each of the two USRPs. Each USRP can only receive data from one antenna at a time. 

The list of antennas that can be connected to each USRP can be seen [here](https://github.com/SETIatHCRO/ATA-Utils/blob/master/RFSwitchUtils/antassign.h#L21).

The IF switch has variable attenuators. We usually set these to 20 dB, but changing this value can be useful to adapt the signal levels in atypical situations.

In [ ]:
att = 20  # Attenuation in dB
ac.rf_switch_thread(antennas)
ac.set_atten_thread([[f'{ant}x', f'{ant}y'] for ant in antennas],
                    [[att, att] for ant in antennas])

## Setting the RFCB LO frequency and focusing feeds

The RFCB is used to tune to a particular frequency from the very wide range (0.5 - 14 GHz) that the ATA feeds offer.

There are four independent LOs in the RFCB, called a, b, c and d. For the GNU Radio backend we typically use LO d (this comes determined by what outputs of the RFCB are connected to the IF switch that goes to the USRPs).

The `set_freq()` function will set the RFCB LO frequency and also focus the feeds appropriately (that's the reason why it needs the list of antennas).

In [ ]:
freq = 1420.405  # Frequency to tune to in MHz
lo = 'd'  # RFCB LO to set
ac.set_freq(freq, antennas, lo)

## Running autotune

Autotune is used to adjust the PAM attenuators in the feeds for optimal signal levels.

In [ ]:
ac.autotune(antennas)

## Pointing the antennas

There are different ways of specifying the direction in which the antennas should point. The cells below give the most usual.

In [ ]:
# Fixed azimuth and elevation
ac.track_source(antennas, azel=[270.0, 35.0])

In [ ]:
# Fixed right ascension and declination
ac.track_source(antennas, radec=[Angle('6h48m56.6s').hour, Angle('2d23m18.4s').deg])

In [ ]:
# Catalogue source
ac.track_source(antennas, source='3c84')

In [ ]:
# Satellite tracking, using a NORAD ID and TLE from Celestrak
norad_id = 43057  # satellite ID

# Download the data from Celestrak in a file in /tmp
url = f'https://celestrak.com/satcat/tle.php?CATNR={norad_id}'
tle_file = f'/tmp/{norad_id}.tle'
with urllib.request.urlopen(url) as f:
    tle = b'\n'.join(f.read().split(b'\n')[1:])
    with open(tle_file, 'wb') as g:
        g.write(tle)

# Use that file for tracking
ac.make_and_track_tle(tle_file, antennas)

## Parking the antennas

At the end of an observation, it is usual to park the antennas at a fixed azimuth and elevation: either (0, 18) or (180, 18).

In [ ]:
ac.park_antennas(antennas)

## Freeing the antennas

When we finish our observations, we should move the antennas back to the 'none' group to indicate that they are free.

In [ ]:
ac.move_ant_group(antennas, 'atagr', 'none')  # Free the antennas